In [1]:
import pandas as pd
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException


In [13]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [14]:
def getInfo(link):
    driver.get(link)
    info, name, end_date = [], [], []
    i = 1


    labels = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="store-active-coupon"]/div/div[5]/div[1]/label'))
                )
    print(len(labels))

    for label in labels: 
        label.click()


    labels = WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="store-active-coupon"]/div/div[5]/label'))
                )
    print(len(labels))
    for label in labels: 
        label.click()

    
    
    screen_height = driver.execute_script("return window.screen.height;")  

    while True:
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(5)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break 

    name1 =  driver.find_elements(By.XPATH, '//*[@id="store-active-coupon"]/div/div[2]')
    name = [s.text for s in name1] + name

    info1 =  driver.find_elements(By.XPATH, '//*[@id="store-active-coupon"]/div/div[3]/p')
    info =  [s.text for s in info1] + info

    end_date1 =  driver.find_elements(By.XPATH, '//*[@id="store-active-coupon"]/div/div[5]/div/div[1]/div[2]')
    end_date =  [s.text for s in end_date1] + end_date

   
    voucher_data = pd.DataFrame(list(zip(name, info, end_date)),
                        columns=['name', 'info', 'end_date']) 
    print("Crawl Complete")
    return voucher_data

url = 'https://iprice.vn/coupons/shopee/'
df = getInfo(url)
driver.close()

8
42
Crawl Complete


In [15]:
# Lấy file CSV
df_unique = df.drop_duplicates(keep='first')
df_unique.to_csv('D://Exam/Shoppe.csv', index=False)
